# Embeddings
https://platform.openai.com/docs/models#embeddings

임베딩(Embeddings)은 텍스트를 수치적으로 표현한 값으로, 두 텍스트 간의 연관성을 측정하는 데 사용된다.

임베딩은 검색, 군집화(clustering), 추천 시스템, 이상 탐지, 분류와 같은 작업에 유용하다.

**모델 및 출력 차원**

| 모델 이름                     | 설명                                                              | 출력 차원 |
|-------------------------------|-------------------------------------------------------------------|-----------|
| **text-embedding-3-large**   | 영어 및 비영어 작업 모두에서 가장 강력한 성능을 가진 모델           | 3,072     |
| **text-embedding-3-small**   | 2세대 ada 임베딩 모델보다 성능이 향상된 모델                        | 1,536     |
| **text-embedding-ada-002**   | 1세대 모델 16개를 대체하는 가장 강력한 2세대 임베딩 모델             | 1,536     |


## MTEB Leaderboard
**Massive Text Embedding Benchmark (MTEB) Leaderboard**

https://huggingface.co/spaces/mteb/leaderboard

**MTEB Leaderboard**는 Hugging Face에서 제공하는 벤치마크 리더보드 페이지로, 다양한 언어 모델(Language Model)과 임베딩 모델(Embedding Model)의 성능을 객관적으로 비교·평가하는 공간이다.

**MTEB Leaderboard에서 순위 산정 방식**

**MTEB Leaderboard**의 순위는 다양한 자연어 처리 태스크(분류, 클러스터링, 검색, 문장 유사도 등)에서 모델이 얻은 점수들의 평균을 기반으로 산정된다. 즉, 여러 벤치마크 데이터셋에서 모델의 성능을 측정하고, 이를 종합하여 평균 점수를 계산한 뒤, 이 평균 점수가 높은 순서대로 모델이 정렬된다.

**주요 평가 방식**

- **평가 태스크 종류**
  - 분류(Classification): F1 점수
  - 클러스터링(Clustering): V-measure
  - 쌍 분류(Pair Classification): Average Precision
  - 재정렬(Reranking): MRR@k, MAP
  - 검색(Retrieval): nDCG@k
  - 의미 유사도(STS): Spearman correlation
  - 요약(Summarization): Spearman correlation  
  각 태스크별로 대표적인 평가 지표가 다르며, 모델은 여러 태스크에서 평가를 받는다[2].

- **평균 점수 산정**
  - 각 태스크별로 모델이 얻은 점수를 모두 합산한 뒤, 태스크 수로 나누어 평균 점수를 구한다.
  - 이 평균 점수가 리더보드의 기본 순위 기준이 된다.

- **부분 평가 가능**
  - 모든 태스크를 수행하지 않아도 특정 태스크만 평가받아 부분 리더보드에 오를 수 있다. 예를 들어, 클러스터링 태스크만 평가받아 해당 부분 순위에 표시될 수 있다.

In [ ]:
from google.colab import userdata
from openai import OpenAI

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=OPENAI_API_KEY)

text = '안녕하세요~'

response = client.embeddings.create(
    model='text-embedding-3-small',
    input=[text]
)
print(response)

print(len(response.data[0].embedding))

CreateEmbeddingResponse(data=[Embedding(embedding=[0.006883355788886547, -0.08132949471473694, -0.006404029205441475, 0.021313322708010674, 0.037833377718925476, -0.03754355013370514, -0.04523507505655289, 0.03580459952354431, -0.04623831808567047, -0.02938384748995304, -0.05408589914441109, 0.003915431443601847, 0.012217259965837002, 0.008639030158519745, -0.0372760184109211, 0.02925008349120617, -0.0442318320274353, 0.019251102581620216, -0.027689483016729355, 0.019429456442594528, 0.06153218448162079, 0.014089979231357574, -0.010690102353692055, -0.01994222402572632, 0.03856908902525902, 0.05578026548027992, 0.05685038864612579, -0.027644895017147064, 0.04447706788778305, -0.05132140964269638, 0.02706524357199669, -0.0386582650244236, -0.014524716883897781, -0.052971187978982925, -0.027310481294989586, 0.004160668235272169, 0.0160630214959383, -0.022394593805074692, -0.025214819237589836, -0.03767731785774231, -0.01445783395320177, -0.005434228572994471, -0.004054770339280367, 0.037

In [ ]:
def texts_to_embedding(texts, model='text-embedding-3-small'):
    # 특수문자/개행문자등을 제거하면 임베딩품질 높아진다.
    texts = [text.replace('\n', ' ') for text in texts]
    response = client.embeddings.create(
        model=model,
        input=texts
    )
    return [data.embedding for data in response.data]

output = texts_to_embedding(['hello world', 'byebye world'])

import numpy as np
np.array(output).shape

(2, 1536)

## 음식리뷰 유사도 검색

In [ ]:
!gdown 1tSQZQFYD64_mrL9CjDcn6KruZp7_smuD

Downloading...
From: https://drive.google.com/uc?id=1tSQZQFYD64_mrL9CjDcn6KruZp7_smuD
To: /content/fine_food_reviews_1k.csv
100% 439k/439k [00:00<00:00, 93.0MB/s]


In [ ]:
# 데이터로드
import pandas as pd

df = pd.read_csv('fine_food_reviews_1k.csv', index_col=0)
df.head()

Time   ProductId          UserId  Score  \
Unnamed: 0                                                  
0           1351123200  B003XPF9BO  A3R7JR3FMEBXQB      5   
1           1351123200  B003JK537S  A3JBPC3WFUT5ZP      1   
2           1351123200  B000JMBE7M   AQX1N6A51QOKG      4   
3           1351123200  B004AHGBX4  A2UY46X0OSNVUQ      3   
4           1351123200  B001BORBHO  A1AFOYZ9HSM2CZ      5   

                                                      Summary  \
Unnamed: 0                                                      
0           where does one  start...and stop... with a tre...   
1                                           Arrived in pieces   
2                   It isn't blanc mange, but isn't bad . . .   
3                 These also have SALT and it's not sea salt.   
4                                      Happy with the product   

                                                         Text  n_tokens  
Unnamed: 0                                                               
0           Wanted to save some to bring to my Chicago fam...        33  
1           Not pleased at all. When I opened the box, mos...        26  
2           I'm not sure that custard is really custard wi...       242  
3           I like the fact that you can see what you're g...       216  
4           My dog was suffering with itchy skin.  He had ...        76

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Time       1000 non-null   int64 
 1   ProductId  1000 non-null   object
 2   UserId     1000 non-null   object
 3   Score      1000 non-null   int64 
 4   Summary    1000 non-null   object
 5   Text       1000 non-null   object
 6   n_tokens   1000 non-null   int64 
dtypes: int64(3), object(4)
memory usage: 62.5+ KB


In [ ]:
# Summary + Text -> Combined
df['combined'] = df['Summary'].str.strip() + '; ' + df['Text'].str.strip()
df.head()

Time   ProductId          UserId  Score  \
Unnamed: 0                                                  
0           1351123200  B003XPF9BO  A3R7JR3FMEBXQB      5   
1           1351123200  B003JK537S  A3JBPC3WFUT5ZP      1   
2           1351123200  B000JMBE7M   AQX1N6A51QOKG      4   
3           1351123200  B004AHGBX4  A2UY46X0OSNVUQ      3   
4           1351123200  B001BORBHO  A1AFOYZ9HSM2CZ      5   

                                                      Summary  \
Unnamed: 0                                                      
0           where does one  start...and stop... with a tre...   
1                                           Arrived in pieces   
2                   It isn't blanc mange, but isn't bad . . .   
3                 These also have SALT and it's not sea salt.   
4                                      Happy with the product   

                                                         Text  n_tokens  \
Unnamed: 0                                                                
0           Wanted to save some to bring to my Chicago fam...        33   
1           Not pleased at all. When I opened the box, mos...        26   
2           I'm not sure that custard is really custard wi...       242   
3           I like the fact that you can see what you're g...       216   
4           My dog was suffering with itchy skin.  He had ...        76   

                                                     combined  
Unnamed: 0                                                     
0           where does one  start...and stop... with a tre...  
1           Arrived in pieces; Not pleased at all. When I ...  
2           It isn't blanc mange, but isn't bad . . .; I'm...  
3           These also have SALT and it's not sea salt.; I...  
4           Happy with the product; My dog was suffering w...

In [ ]:
# 임베딩변환
df['embedding'] = texts_to_embedding(df['combined'].tolist())
df.head()

Time   ProductId          UserId  Score  \
Unnamed: 0                                                  
0           1351123200  B003XPF9BO  A3R7JR3FMEBXQB      5   
1           1351123200  B003JK537S  A3JBPC3WFUT5ZP      1   
2           1351123200  B000JMBE7M   AQX1N6A51QOKG      4   
3           1351123200  B004AHGBX4  A2UY46X0OSNVUQ      3   
4           1351123200  B001BORBHO  A1AFOYZ9HSM2CZ      5   

                                                      Summary  \
Unnamed: 0                                                      
0           where does one  start...and stop... with a tre...   
1                                           Arrived in pieces   
2                   It isn't blanc mange, but isn't bad . . .   
3                 These also have SALT and it's not sea salt.   
4                                      Happy with the product   

                                                         Text  n_tokens  \
Unnamed: 0                                                                
0           Wanted to save some to bring to my Chicago fam...        33   
1           Not pleased at all. When I opened the box, mos...        26   
2           I'm not sure that custard is really custard wi...       242   
3           I like the fact that you can see what you're g...       216   
4           My dog was suffering with itchy skin.  He had ...        76   

                                                     combined  \
Unnamed: 0                                                      
0           where does one  start...and stop... with a tre...   
1           Arrived in pieces; Not pleased at all. When I ...   
2           It isn't blanc mange, but isn't bad . . .; I'm...   
3           These also have SALT and it's not sea salt.; I...   
4           Happy with the product; My dog was suffering w...   

                                                    embedding  
Unnamed: 0                                                     
0           [0.030276387929916382, -0.020651785656809807, ...  
1           [0.01129516027867794, 0.03488067910075188, -0....  
2           [0.0022991024889051914, 0.004840383306145668, ...  
3           [-0.015768831595778465, 0.013766037300229073, ...  
4           [7.225894660223275e-05, -0.06748031824827194, ...

In [ ]:
# embed_df로 변환
embed_df = pd.DataFrame(df['embedding'].tolist(), index=df['combined'])
embed_df.head()

0         1     \
combined                                                                 
where does one  start...and stop... with a trea...  0.030276 -0.020652   
Arrived in pieces; Not pleased at all. When I o...  0.011295  0.034881   
It isn't blanc mange, but isn't bad . . .; I'm ...  0.002299  0.004840   
These also have SALT and it's not sea salt.; I ... -0.015769  0.013766   
Happy with the product; My dog was suffering wi...  0.000072 -0.067480   

                                                        2         3     \
combined                                                                 
where does one  start...and stop... with a trea... -0.028664  0.022054   
Arrived in pieces; Not pleased at all. When I o... -0.038722  0.023746   
It isn't blanc mange, but isn't bad . . .; I'm ... -0.043847 -0.036250   
These also have SALT and it's not sea salt.; I ... -0.027760  0.039194   
Happy with the product; My dog was suffering wi...  0.014100  0.030484   

                                                        4         5     \
combined                                                                 
where does one  start...and stop... with a trea... -0.028745 -0.055652   
Arrived in pieces; Not pleased at all. When I o... -0.018568 -0.055954   
It isn't blanc mange, but isn't bad . . .; I'm ...  0.010121 -0.053840   
These also have SALT and it's not sea salt.; I ...  0.014032  0.008227   
Happy with the product; My dog was suffering wi... -0.019324  0.005436   

                                                        6         7     \
combined                                                                 
where does one  start...and stop... with a trea... -0.022748  0.057457   
Arrived in pieces; Not pleased at all. When I o...  0.009249 -0.038861   
It isn't blanc mange, but isn't bad . . .; I'm ...  0.042572  0.004611   
These also have SALT and it's not sea salt.; I ... -0.011218 -0.026797   
Happy with the product; My dog was suffering wi...  0.006728  0.068204   

                                                        8         9     ...  \
combined                                                                ...   
where does one  start...and stop... with a trea... -0.026504 -0.016557  ...   
Arrived in pieces; Not pleased at all. When I o... -0.004861  0.013188  ...   
It isn't blanc mange, but isn't bad . . .; I'm ...  0.046371 -0.009974  ...   
These also have SALT and it's not sea salt.; I ... -0.008860 -0.047129  ...   
Happy with the product; My dog was suffering wi...  0.013150  0.045205  ...   

                                                        1526      1527  \
combined                                                                 
where does one  start...and stop... with a trea...  0.026262 -0.022554   
Arrived in pieces; Not pleased at all. When I o...  0.037163  0.051054   
It isn't blanc mange, but isn't bad . . .; I'm ...  0.004311 -0.016213   
These also have SALT and it's not sea salt.; I ...  0.015490  0.020066   
Happy with the product; My dog was suffering wi...  0.013501  0.010725   

                                                        1528      1529  \
combined                                                                 
where does one  start...and stop... with a trea... -0.011849  0.018669   
Arrived in pieces; Not pleased at all. When I o... -0.004009  0.012005   
It isn't blanc mange, but isn't bad . . .; I'm ... -0.010114 -0.004264   
These also have SALT and it's not sea salt.; I ...  0.048498  0.014932   
Happy with the product; My dog was suffering wi... -0.006021 -0.011103   

                                                        1530      1531  \
combined                                                                 
where does one  start...and stop... with a trea... -0.000274  0.047913   
Arrived in pieces; Not pleased at all. When I o... -0.002236  0.035159   
It isn't blanc mange, but isn't bad . . .; I'm ... -0.005114  0.011988   
These also have SA

In [ ]:
# embed_df로 변환
embed_df = df[['embedding']]
# embed_df = df['embedding'].to_frame()
embed_df.index = df['combined']
embed_df.head()

embedding
combined                                                                                             
where does one  start...and stop... with a trea...  [0.030276387929916382, -0.020651785656809807, ...
Arrived in pieces; Not pleased at all. When I o...  [0.01129516027867794, 0.03488067910075188, -0....
It isn't blanc mange, but isn't bad . . .; I'm ...  [0.0022991024889051914, 0.004840383306145668, ...
These also have SALT and it's not sea salt.; I ...  [-0.015768831595778465, 0.013766037300229073, ...
Happy with the product; My dog was suffering wi...  [7.225894660223275e-05, -0.06748031824827194, ...

In [ ]:
# 유사도 검색
from sklearn.metrics.pairwise import cosine_similarity

def review_search(query, emb_df, top_n=5):
    # 검색어 동일한 임베딩변환
    query_emb = texts_to_embedding([query])

    # cosine_similarity(X, Y): X, Y 모두 batch차원 감싸서 전달
    embed_df['cos_sim'] = embed_df['embedding'].apply(lambda emb: cosine_similarity([emb], query_emb)[0, 0])
    top_n_texts = embed_df.sort_values('cos_sim', ascending=False).head(top_n)
    return top_n_texts

review_search('delicious fruit', embed_df)

/tmp/ipython-input-32-3991446176.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  embed_df['cos_sim'] = embed_df['embedding'].apply(lambda emb: cosine_similarity([emb], query_emb)[0, 0])


embedding  \
combined                                                                                                
Delicious .; These plums are sweet and juicy, a...  [0.03158695250749588, -0.001961484318599105, -...   
very delicious; This is my very favorite of the...  [-0.005982326343655586, -0.042343031615018845,...   
Delicious!!!!; A coffee treat. Now that my husb...  [-0.01914854347705841, -0.044288646429777145, ...   
Delicious!; For anyone who says "I don't like f...  [-0.016479600220918655, 0.0018902381416410208,...   
yummy; these are the best super yummy and i nev...  [-0.022759903222322464, -0.05974780395627022, ...   

                                                     cos_sim  
combined                                                      
Delicious .; These plums are sweet and juicy, a...  0.541842  
very delicious; This is my very favorite of the...  0.500116  
Delicious!!!!; A coffee treat. Now that my husb...  0.496556  
Delicious!; For anyone who says "I don't like f...  0.493553  
yummy; these are the best super yummy and i nev...  0.482908

In [ ]:
review_search('bad delivery', embed_df)

/tmp/ipython-input-32-3991446176.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  embed_df['cos_sim'] = embed_df['embedding'].apply(lambda emb: cosine_similarity([emb], query_emb)[0, 0])


embedding  \
combined                                                                                                
great product, poor delivery; The coffee is exc...  [-0.032881032675504684, -0.010334637016057968,...   
great product, poor delivery; The coffee is exc...  [-0.032881032675504684, -0.010334637016057968,...   
great product, poor delivery; The coffee is exc...  [-0.032881032675504684, -0.010334637016057968,...   
great product, poor delivery; The coffee is exc...  [-0.032881032675504684, -0.010334637016057968,...   
Disappointed; The metal cover has severely disf...  [0.007573779672384262, -0.008688628673553467, ...   

                                                     cos_sim  
combined                                                      
great product, poor delivery; The coffee is exc...  0.511626  
great product, poor delivery; The coffee is exc...  0.511626  
great product, poor delivery; The coffee is exc...  0.511626  
great product, poor delivery; The coffee is exc...  0.511626  
Disappointed; The metal cover has severely disf...  0.377055